In [130]:
import numpy as np
from PIL import Image, ImageFilter

In [131]:
# Read the video from specified path
cam = cv2.VideoCapture("./new/encryptedvideo.avi")
# frames = cam.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cam.get(cv2.CAP_PROP_FPS)
# seconds = round(frames / fps)
print(fps)

30.0


In [132]:
image='./SquareDetectionResults/'
ext='.bmp'

In [133]:
import cv2

In [134]:
from PIL import Image
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [135]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color=0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [136]:
def LogisticDecryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1
    
    L_x = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L_x *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    
    C = round((L_x*L_y*10**4) % 256)
    I_prev = C
    I_prev_r = C
    I_prev_g = C
    I_prev_b = C
    I = C
    I_r = C
    I_g = C
    I_b = C
    x_prev = 4*(S_x)*(1-S_x)
    y_prev = 4*(L_x)*(1-S_y)
    x = x_prev
    y = y_prev
    imageMatrix, dimension, color = getImageMatrix(imageName)
    henonDecryptedImage = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
                I_r = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_r + key_list[7]) % N) ^ imageMatrix[i][j][0]) + N-key_list[6])%N
                I_g = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_g + key_list[7]) % N) ^ imageMatrix[i][j][1]) + N-key_list[6])%N
                I_b = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev_b + key_list[7]) % N) ^ imageMatrix[i][j][2]) + N-key_list[6])%N
                I_prev_r = imageMatrix[i][j][0]
                I_prev_g = imageMatrix[i][j][1]
                I_prev_b = imageMatrix[i][j][2]
                row.append((I_r,I_g,I_b))
                x = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j][0]/256 + key_list[8]/256 + key_list[9]/256) % 1  
            else:
                I = ((((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((I_prev+key_list[7]) % N) ^ imageMatrix[i][j]) + N-key_list[6])%N
                I_prev = imageMatrix[i][j]
                row.append(I)
                x = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
                y = (x +  imageMatrix[i][j]/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        henonDecryptedImage.append(row)
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels
    pix = im.load()
    for x in range(dimension-2):
        for y in range(dimension-2):
            pix[x, y] = henonDecryptedImage[x][y]
    im.save(image+"decryp"+imageName[28:-4]+ext, "BMP")

In [137]:
key = "abcdefghijklm"

In [144]:
folder_dir = "./SquareDetectionResults/encrypted_frames"
cropped_images = []
for images in os.listdir(folder_dir):
# Open image and make into Numpy array
    im = Image.open('./SquareDetectionResults/encrypted_frames/'+images).convert('RGB')
    na = np.array(im)
    orig = na.copy()    # Save original
    # Median filter to remove outliers
    im = im.filter(ImageFilter.MedianFilter(3))
    # Find X,Y coordinates of all yellow pixels
    yellowY, yellowX = np.where(np.all(na==[0, 0, 255],axis=2))
    print(yellowX)
    #yellowY2, yellowX2= np.where(np.all(na==[0, 0, 255],axis=-2))
    zipped = np.column_stack((yellowX,yellowY))
    print("xipp",zipped)
    uniqueY = list(yellowY)
    uniqueX= list(set(yellowX))

    uniquesetX={uniqueX[0]}
    uniquesetY={uniqueY[0]}

    print(uniquesetX)
    print(uniquesetY)

    print(uniqueX)
    print(uniqueY)
    for x in range(0,len(uniqueX)-1):
      if(uniqueX[x+1]-uniqueX[x]>=2):
        uniquesetX.add(uniqueX[x])
        uniquesetX.add(uniqueX[x+1])
    uniquesetX.add(uniqueX[len(uniqueX)-1])
    uniquesetX=sorted(uniquesetX)
    uniquesetY=list()
    for a in uniquesetX:
      for x,y in zipped:
        if(a==x):
          uniquesetY.append(y)
    uniqueset2Y=list()
    uniqueset2Y.append(uniquesetY[0])
    print(uniqueset2Y)
    for x in range(0,len(uniquesetY)-1):
      if(uniquesetY[x+1]-uniquesetY[x]!=1):
        uniqueset2Y.append(uniquesetY[x])
        uniqueset2Y.append(uniquesetY[x+1])
    uniqueset2Y.append(uniquesetY[len(uniquesetY)-1])
    print("uniqesetY",uniqueset2Y)
    uniqueset3Y=list()
    for x in range(0,len(uniqueset2Y),4):
      uniqueset3Y.append(uniqueset2Y[x])
      uniqueset3Y.append(uniqueset2Y[x+1])
    print("tellmefinal")
    print(uniqueset3Y)
    img = cv2.imread('./SquareDetectionResults/encrypted_frames/'+images)
    faces=[]
    print(img)
    for i in range(0,len(uniquesetX)-1,2):
        a=uniqueset3Y[i]
        b=uniqueset3Y[i+1]
        c=uniquesetX[i]
        d=uniquesetX[i+1]
        face=img[a+1:b,c+1:d]
        filename=image+ext
        cv2.imwrite(filename, face)
        LogisticDecryption(filename, key)
        filename=image+"decryp"+ext
        DecryptResult=cv2.imread(filename)
        img[a+1:b,c+1:d]=DecryptResult
        cv2.imwrite('./decrypted_frames/decrypt'+images, img)

[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
{320}
{161}
[320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 49

[ 320  321  322 ... 1016 1017 1018]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1016  409]
 [1017  409]
 [1018  409]]
{320}
{161}
[320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 49

[ 319  320  321 ... 1018 1019 1020]
xipp [[ 319  160]
 [ 320  160]
 [ 321  160]
 ...
 [1018  410]
 [1019  410]
 [1020  410]]
{319}
{160}
[319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 49

[ 320  321  322 ... 1020 1021 1022]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1020  411]
 [1021  411]
 [1022  411]]
{320}
{161}
[320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 49

[ 320  321  322 ... 1019 1020 1021]
xipp [[ 320  161]
 [ 321  161]
 [ 322  161]
 ...
 [1019  410]
 [1020  410]
 [1021  410]]
{320}
{161}
[320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 49

...

[ 318  319  320 ... 1094 1095 1096]
xipp [[ 318  159]
 [ 319  159]
 [ 320  159]
 ...
 [1094  390]
 [1095  390]
 [1096  390]]
{318}
{159}
[318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49

[ 318  319  320 ... 1092 1093 1094]
xipp [[ 318  159]
 [ 319  159]
 [ 320  159]
 ...
 [1092  391]
 [1093  391]
 [1094  391]]
{318}
{159}
[318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49

[ 318  319  320 ... 1087 1088 1089]
xipp [[ 318  158]
 [ 319  158]
 [ 320  158]
 ...
 [1087  393]
 [1088  393]
 [1089  393]]
{318}
{158}
[318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49

[ 318  319  320 ... 1086 1087 1088]
xipp [[ 318  158]
 [ 319  158]
 [ 320  158]
 ...
 [1086  396]
 [1087  396]
 [1088  396]]
{318}
{158}
[318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49

[ 318  319  320 ... 1086 1087 1088]
xipp [[ 318  158]
 [ 319  158]
 [ 320  158]
 ...
 [1086  396]
 [1087  396]
 [1088  396]]
{318}
{158}
[318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49

In [151]:
# importing libraries
import os
import cv2
from PIL import Image
import os.path
import shutil
# Checking the current directory path
print(os.getcwd())

# Folder which contains all the images
# from which video is to be generated

os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/decrypted_frames")
path = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/decrypted_frames"

mean_height = 0
mean_width = 0
num_of_images = len(os.listdir('.'))
print(num_of_images)

for file in os.listdir('.'):
	im = Image.open(os.path.join(path, file))
	width, height = im.size
	mean_width += width
	mean_height += height
	# im.show() # uncomment this for displaying the image

# Finding the mean height and width of all images.
# This is required because the video frame needs
# to be set with same width and height. Otherwise
# images not equal to that width height will not get
# embedded into the video
mean_width = int(mean_width / num_of_images)
mean_height = int(mean_height / num_of_images)

# print(mean_height)
# print(mean_width)

# Resizing of the images to give
# them same width and height
for file in os.listdir('.'):
	if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith("png"):
		# opening image using PIL Image
		im = Image.open(os.path.join(path, file))

		# im.size includes the height and width of image
		width, height = im.size
		print(width, height)

		# resizing
		imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)
		imResize.save( file, 'JPEG', quality = 95) # setting quality
		# printing each resized image name
		print(im.filename.split('\\')[-1], " is resized")


# Video Generating function
def generate_video():
	image_folder = "C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/decrypted_frames" # make sure to use your folder
	video_name = 'decryptedvideo.avi'
	os.chdir("C:/Users/USER/Downloads/Secure-Sharing-of-CCTV-footage-master (2)/Secure-Sharing-of-CCTV-footage-master/new")
	
	images = [img for img in os.listdir(image_folder)
			if img.endswith(".jpg") or
				img.endswith(".jpeg") or
				img.endswith("png")]
	
	# Array images should only consider
	# the image files ignoring others if any
	print(images)

	frame = cv2.imread(os.path.join(image_folder, images[0]))

	# setting the frame width, height width
	# the width, height of first image
	height, width, layers = frame.shape

	video = cv2.VideoWriter(video_name, 0, fps, (width, height))

	# Appending the images to the video one by one
	for image in images:
		video.write(cv2.imread(os.path.join(image_folder, image)))
	# Deallocating memories taken for window creation
	cv2.destroyAllWindows()
	video.release() # releasing the video generated


# Calling the generate_video function
generate_video()

C:\Users\USER\Downloads\Secure-Sharing-of-CCTV-footage-master (2)\Secure-Sharing-of-CCTV-footage-master\new
61
1920 1080
decryptEncryptionResult00.png  is resized
1920 1080
decryptEncryptionResult01.png  is resized
1920 1080


C:\Users\USER\AppData\Local\Temp\ipykernel_17960\1438706589.py:51: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  imResize = im.resize((mean_width, mean_height), Image.ANTIALIAS)


decryptEncryptionResult02.png  is resized
1920 1080
decryptEncryptionResult03.png  is resized
1920 1080
decryptEncryptionResult04.png  is resized
1920 1080
decryptEncryptionResult05.png  is resized
1920 1080
decryptEncryptionResult06.png  is resized
1920 1080
decryptEncryptionResult07.png  is resized
1920 1080
decryptEncryptionResult08.png  is resized
1920 1080
decryptEncryptionResult09.png  is resized
1920 1080
decryptEncryptionResult10.png  is resized
1920 1080
decryptEncryptionResult11.png  is resized
1920 1080
decryptEncryptionResult12.png  is resized
1920 1080
decryptEncryptionResult13.png  is resized
1920 1080
decryptEncryptionResult14.png  is resized
1920 1080
decryptEncryptionResult15.png  is resized
1920 1080
decryptEncryptionResult16.png  is resized
1920 1080
decryptEncryptionResult17.png  is resized
1920 1080
decryptEncryptionResult18.png  is resized
1920 1080
decryptEncryptionResult19.png  is resized
1920 1080
decryptEncryptionResult20.png  is resized
1920 1080
decryptEncry